# 实验五 RLC电路暂态特性的研究

Copyright © Zjl37 2024

In [ ]:
import numpy as np
import yaml
import re
import scipy
import matplotlib.pyplot as plt

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 从文件中读取数据
data_file = './data.yaml'
data = yaml.safe_load(open(data_file))
def data_to_list(ln, cast=float):
    return list(map(cast, re.split(r'[,，;；]?\s+', ln)))

In [ ]:
def parse_v(s: str, unit='V'):
    suffixes = [
        ('µ'+unit, 1e-6),
        ('u'+unit, 1e-6),
        ('m'+unit, 1e-3),

        ('k'+unit, 1e3),
        (unit, 1),
    ]
    for suffix, factor in suffixes:
        if s.endswith(suffix):
            return float(s[:-len(suffix)])*factor
    
    try:
        res = float(s)
        if unit != '':
            print(f"警告⚠️： 数值 {s} 未写明单位。")
        return res
    except:
        raise ValueError(f"无法解析数值 {s}！")

### 3. RLC 串联电路的暂态特性

In [ ]:
st = data['st3']
st

UC = data_to_list(st['UC(t1+nT)'], cast=parse_v) # (V)
UC = np.array(UC)

if UC[0] == 114:
    raise ValueError(f"请先在 {data_file} 中填写好你的实验数据。")

In [ ]:

E = st['E'] # (V)
T = st['T'] # (µs)
t1 = st['t1'] # (µs)

n = len(UC)
t = np.linspace(t1, t1+(n-1)*T, n) # (µs)

y = np.log(UC/E)

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(t, y) # slope: (1/µs)
print(f"拟合直线的斜率为 {slope} µs⁻¹")


In [ ]:
tau = -1/slope # (µs)
print(f"实验测得的时间常数 τ = {tau} µs")

In [ ]:
def uhat(t):
    return intercept + slope*t

plt.figure(figsize=(5, 3))
plt.plot(t, uhat(t), '--', label="拟合直线")
plt.plot(t, y, 'o', label=r"$ \ln (U_C/E - 1) $")
plt.xlabel(r"$t \: / \: \rm{µs}$")
plt.grid()
# 如图例上中文不能显示，在下方修改字体
plt.legend(prop={'family': 'Source Han Sans CN'})
plt.savefig('./fig/st3-UC-E.svg', bbox_inches='tight')
plt.show()

In [ ]:
RL = 6.5 # (Ω)
RS = 50 # (Ω)
L = 10 # (mH)

tau0 = 2*L/(RL+RS) # (ms)
print(f"时间常数理论值 τ₀ = {tau0} ms")